**Step 1: Install PySpark**

In [ ]:
# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=17b5dd2abe70b253e9e3de663e0397476738de532f7b1bc1946fda1caf9e7eef
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


**Step 2: Create a connection with the SQL Server from here at PySpark**

note1: This step assumes you have SQL Server and it is operational

note2: Create a database. For example, it can be called 'user_database'

note3: Create tables in that database. For example, a table called 'user_table

note4: Populate the table with data of your choice. For example, adding data about employees.


**Step 4: Make sure to replace the names with the correct ones**

user_name = 'my_user' should be exactly the name you registered SQL Server.

Password = 'user1234' should be exactly your password you set in the SQL Server.

server_name = 'my_server' should be exactly your SQL server name indicated in your connection

PORT = '1433' should be exactly the port number to access.

database_name = 'my_database' should be exactly the name of your database how you named it SQL server.

In [ ]:
# connection setup  -----> add username, password, server_name, and database_name
username = "dbo"
password = "user1234"
server_name = "myPass"
port = "1433"
database_name = "my_database"

jdbc_url = "jdbc:sqlserver://" + server_name + ":" + port +";database=" + database_name

connectionProperties = {
            "user": username,
            "password": password,
            }


**Step 5: Now the connection has been established. Proceed with executing the commands**

In [ ]:
# start the pyspark session

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Create a Spark session
spark = SparkSession.builder.appName("SQL Database Connection").getOrCreate()

In [ ]:
# define the data types to be used in the database

from pyspark.sql.types import StringType, StructField, StructType, IntegerType, FloatType, DoubleType, BooleanType, DateType, TimestampType
from pyspark.sql.functions import col, when
import datetime

In [ ]:
# now, execute the code and get to create the tables and modify them

# Prompt the user to create a table in the SQL database
create_table = input("Do you want to create a table in the SQL database or edit/view an existing table? (C/E)")

if create_table.lower() != "c" and create_table.lower() != "e":
    print("No table created.")
        
else:
        if create_table.lower() == "c":
            # Prompt the user for the name of the table and the column names
            table_name = input("Enter the name of the table:")
            # take column names and data types from user
            column_data = input(
            "Enter column names and data types (separated by commas, e.g. col1:int,col2:string):").split(",")
            
            fields = []
            for col_data in column_data:
                col_name, col_type = col_data.split(":")
                if col_type == "int":
                    fields.append(StructField(col_name, IntegerType(), True))
                elif col_type == "float":
                    fields.append(StructField(col_name, FloatType(), True))
                elif col_type == "double":
                    fields.append(StructField(col_name, DoubleType(), True))
                elif col_type == "boolean":
                    fields.append(StructField(col_name, BooleanType(), True))
                elif col_type == "date":
                    fields.append(StructField(col_name, DateType(), True))
                elif col_type == "timestamp":
                    fields.append(StructField(col_name, TimestampType(), True))
                else:
                    fields.append(StructField(col_name, StringType(), True))

            # create DataFrame schema based on column names and data types
            schema = StructType(fields)

            # create empty DataFrame with schema
            df = spark.createDataFrame([], schema)

            # Write the DataFrame to the SQL database
            df.write.format("jdbc").options(
                url=jdbc_url,
                dbtable=table_name,
                user=username,
                password=password).mode("overwrite").save()

            print(f"Table {table_name} created in the SQL database.")
        else: 
            print("Choose one of the following tables by writing its name:")
            # Retrieve table names
            table_names = spark.read.jdbc(
                url=jdbc_url, table="INFORMATION_SCHEMA.TABLES", properties=connectionProperties).\
                            filter("TABLE_TYPE = 'BASE TABLE'").select("TABLE_NAME").collect()

            # Print table names
            for tableName in table_names:
                print(tableName.TABLE_NAME)
                
            table_name = input("Enter table name: ")
            

        # Prompt the user to add or delete entries in the table
        while True:
            df = spark.read.jdbc(url=jdbc_url,
                                     table=table_name, properties=connectionProperties).persist()

            schema = df.schema
            
            # create an empty list to store the column data
            column_data = []

            # loop through each column in the schema
            for field in schema.fields:
                # get the name and data type of the column
                col_name = field.name
                col_type = field.dataType

                # map the data type to a string representation
                if isinstance(col_type, IntegerType):
                    data_type_str = "int"
                elif isinstance(col_type, FloatType):
                    data_type_str = "float"
                elif isinstance(col_type, DoubleType):
                    data_type_str = "double"
                elif isinstance(col_type, BooleanType):
                    data_type_str = "boolean"
                elif isinstance(col_type, DateType):
                    data_type_str = "date"
                elif isinstance(col_type, TimestampType):
                    data_type_str = "timestamp"
                else:
                    data_type_str = "string"

                # append the column name and data type string to the list
                column_data.append(f"{col_name}:{data_type_str}")
            
            # ask user if they want to add or delete an entry
            modify_entry = input(
                "Do you want to add, delete, update an entry (A/D/U), print the table (P)," +
                "or switch/create a table (S)?")

            if modify_entry.lower() == "a":

                # create empty dictionary to store entry data
                entry_data = {}

                # take input for each column in the table
                for col_data in column_data:
                    col_name, col_type = col_data.split(":")
                    entry_value = input(f"Enter value for {col_name}: ")
                    if col_type == "int":
                        entry_data[col_name] = int(entry_value)
                    elif col_type == "float" or col_type == "double":
                        entry_data[col_name] = float(entry_value)
                    elif col_type == "boolean":
                        entry_data[col_name] = bool(entry_value)
                    elif col_type == "date":
                        entry_data[col_name] = datetime.strptime(entry_value, "%Y-%m-%d").date()
                    elif col_type == "timestamp":
                        entry_data[col_name] = datetime.strptime(entry_value, "%Y-%m-%d %H:%M:%S")
                    else:
                        entry_data[col_name] = entry_value

                # create DataFrame with new entry
                new_entry = spark.createDataFrame([entry_data], schema)

                # append it to original DataFrame
                df = df.union(new_entry)

                print(f"New row added to table {table_name}.")
                
            elif modify_entry.lower() == "d":
                # take column name and value to delete from user
                print("Column names:", df.columns)
                delete_col = input("Enter column name to delete entry from: ")
                delete_value = input("Enter value to delete: ")

                # search for column type
                for col_data in column_data:
                    col_name, col_type = col_data.split(":")

                    if delete_col != col_name:
                      continue

                    if col_type == "int":
                        delete_value = int(delete_value)
                    elif col_type == "float" or col_type == "double":
                        delete_value = float(delete_value)
                    elif col_type == "boolean":
                        delete_value = bool(delete_value)
                    elif col_type == "date":
                        delete_value = datetime.strptime(delete_value, "%Y-%m-%d").date()
                    elif col_type == "timestamp":
                        delete_value = datetime.strptime(delete_value, "%Y-%m-%d %H:%M:%S")

                if delete_col in df.columns:
                    # delete entry from DataFrame
                    df = df.filter(f"{delete_col} != '{delete_value}'")

                else:
                    print("Column is not found.")

            elif modify_entry.lower() == "u":

                # take column name and value to update from user
                print("Column names:", df.columns)
                update_col = input("Enter column name to update entry: ")
                update_value = input("Enter value to update: ")
                update_col_new_value = input("Enter new value: ")

                # search for column type
                for col_data in column_data:
                    col_name, col_type = col_data.split(":")

                    if update_col != col_name:
                      continue

                    if col_type == "int":
                        update_col_new_value = int(update_col_new_value)
                        update_value = int(update_value)
                    elif col_type == "float":
                        update_col_new_value = float(update_col_new_value)
                        update_value = float(update_value)
                    elif col_type == "boolean":
                        update_col_new_value = bool(update_col_new_value)
                        update_value = bool(update_value)
                    elif col_type == "date":
                        update_col_new_value = datetime.strptime(update_col_new_value, "%Y-%m-%d").date()
                        update_value = datetime.strptime(update_value, "%Y-%m-%d").date()
                    elif col_type == "timestamp":
                        update_col_new_value = datetime.strptime(update_col_new_value, "%Y-%m-%d %H:%M:%S")
                        update_value = datetime.strptime(update_value, "%Y-%m-%d %H:%M:%S")

                # update the entry in the DataFrame
                df = df.withColumn(update_col, when(col(update_col) == update_value, 
                                                    update_col_new_value).otherwise(col(update_col)))

            elif modify_entry.lower() == "p":
                # show the contents of the DataFrame
                df.show()

            elif modify_entry.lower() == "s":
                
                print("Choose one of the following tables by writing its name:")
                # Retrieve table names
                table_names = spark.read.jdbc(
                url=jdbc_url, table="INFORMATION_SCHEMA.TABLES", properties=connectionProperties).\
                            filter("TABLE_TYPE = 'BASE TABLE'").select("TABLE_NAME").collect()

                # Store table names in a list
                tables = [tableName.TABLE_NAME for tableName in table_names]
                
                # Print table names
                print(tables)
                
                switch_table = input("Enter table name to switch to or enter 'C' to create a new table: ")
                
                # check if the user wants to create new table
                if switch_table.lower() == "c":
                    # take table name
                    table_name = input("Enter table name: ")

                    # take column names and data types from user
                    column_data = input("Enter column names and data types (separated by commas, e.g." + 
                                        "col1:int,col2:string):").split(",")

                    fields = []
                    for col_data in column_data:
                        col_name, col_type = col_data.split(":")
                        if col_type == "int":
                            fields.append(StructField(col_name, IntegerType(), True))
                        elif col_type == "float" or col_type == "double":
                            fields.append(StructField(col_name, FloatType(), True))
                        elif col_type == "boolean":
                            fields.append(StructField(col_name, BooleanType(), True))
                        elif col_type == "date":
                            fields.append(StructField(col_name, DateType(), True))
                        elif col_type == "timestamp":
                            fields.append(StructField(col_name, TimestampType(), True))
                        else:
                            fields.append(StructField(col_name, StringType(), True))

                    # create DataFrame schema based on column names and data types
                    schema = StructType(fields)
                    
                    # create empty DataFrame with schema
                    df = spark.createDataFrame([], schema)

                # check if table exists in dictionary
                elif switch_table in tables:
                    # switch to selected table
                    table_name = switch_table

                else:
                    print("Table does not exist.")
                    continue

            else:
              print("Invalid input.")

            if modify_entry.lower() != "p" and modify_entry.lower() != "s":
                
                # Write the updated table to the SQL database
                df.write.format("jdbc").options(url=jdbc_url, 
                                     dbtable=table_name, user=username,
                                password=password, truncate = True).mode("overwrite").save()


            # ask user if they want to continue modifying the table
            continue_modifying = input("Do you want to continue modifying the table? (Y/N) ")

            if continue_modifying.lower() != "y":
              break

Do you want to create a table in the SQL database or edit/view an existing table? (C/E)c
Enter the name of the table:bd
Enter column names and data types (separated by commas, e.g. col1:int,col2:string):name:string,id:int


Py4JJavaError: ignored